In [135]:
import random

def singlePointCrossover(firstParent: list, secondParent: list):

   crossOverPoint = random.randint(1, len(firstParent) - 1)
   print(crossOverPoint)
   crossOverPart1 = firstParent[crossOverPoint: len(firstParent)]
   crossOverPart2 = secondParent[crossOverPoint: len(secondParent)]

   print(crossOverPart1)
   print(crossOverPart2)

   firstOffSpring = firstParent
   secondOffSpring = secondParent

   firstOffSpring[crossOverPoint: len(firstParent)] = crossOverPart2
   secondOffSpring[crossOverPoint: len(secondParent)] = crossOverPart1

   print(firstOffSpring)
   print(secondOffSpring)




In [138]:
firstParent = [0, 0 , 1, 1, 0, 1]
secondParent = [1, 1, 0, 0, 0, 1]

singlePointCrossover(firstParent, secondParent)

1
[0, 1, 1, 0, 1]
[1, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 1]
[1, 0, 1, 1, 0, 1]


In [139]:
def multiplePointCrossover(firstParent: list, secondParent: list):
    firstCrossOverPoint = random.randint(1, len(firstParent) - 1)
    secondCrossOverPoint = random.randint(1, len(secondParent) - 1)

    while (firstCrossOverPoint == secondCrossOverPoint):
        secondCrossOverPoint = random.randint(1, len(firstParent) - 1)
    
    if (firstCrossOverPoint > secondCrossOverPoint):
        firstCrossOverPoint, secondCrossOverPoint = secondCrossOverPoint, firstCrossOverPoint
    
    print(firstCrossOverPoint)
    print(secondCrossOverPoint)

    crossOverPart1 = firstParent[firstCrossOverPoint: secondCrossOverPoint + 1]
    crossOverPart2 = secondParent[firstCrossOverPoint: secondCrossOverPoint + 1]

    print(crossOverPart1)
    print(crossOverPart2)

    firstOffSpring = firstParent
    secondOffSpring = secondParent

    firstOffSpring[firstCrossOverPoint: secondCrossOverPoint + 1] = crossOverPart2
    secondOffSpring[firstCrossOverPoint: secondCrossOverPoint + 1] = crossOverPart1

    print(firstOffSpring)
    print(secondOffSpring)

    


In [141]:
firstParent = [0, 0 , 1, 1, 0, 1]
secondParent = [1, 1, 0, 0, 0, 1]

multiplePointCrossover(firstParent, secondParent)

1
4
[0, 1, 1, 0]
[1, 0, 0, 0]
[0, 1, 0, 0, 0, 1]
[1, 0, 1, 1, 0, 1]
